In [14]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_json("./data/CanucksPP-2022_10_19.json", lines=True)

In [15]:
df.head()

,atp_last,attendance_current,attended_last,attended_prior,clientcode,dimcustomermasterid,distance,events_current,events_last,events_prior,lkupclientid,product_current,product_last,product,sascore,seasonyear,scoredate,spend_current,tenure,volume_current,sends,sends_prior,opens,opens_prior,date_last_send,date_last_touch,date_last_save,newflag,priority
0,31.0,1.0,1,3,CANUCKS,10523,5.65,2,1,3,7,None,Individual,None,0.6600,2022,2022-10-19 19:53:12,208.5,-1,4,7,3,5,1,2022-10-13,NaN,2022-10-18,0,0.33
1,0.0,0.0,1,1,CANUCKS,10547,250.55,0,1,1,7,None,None,None,0.9600,2022,2022-10-19 19:53:12,0.0,-1,0,2,0,0,0,2022-08-29,NaN,2022-10-18,0,0.63
2,130.5,0.0,1,2,CANUCKS,10556,82.33,0,1,2,7,None,Individual,Groups,0.4900,2022,2022-10-19 19:53:12,0.0,-1,0,0,0,0,0,None,NaN,2022-10-18,0,0.16
3,0.0,0.0,0,0,CANUCKS,10559,250.55,0,1,1,7,None,None,None,0.9500,2022,2022-10-19 19:53:12,0.0,-1,0,0,0,0,0,None,NaN,2022-09-29,0,0.62
4,0.0,0.0,0,2,CANUCKS,10572,18.60,0,0,2,7,None,None,None,0.7015,2022,2022-10-19 19:53:12,0.0,-1,0,12,9,0,0,2022-10-12,NaN,2022-10-05,0,0.37


In [28]:
df["product"].value_counts()

None           329780
Individual      50488
Groups           4877
Quarter          1299
Mini Pack         753
Full Season       319
Half Season       248
Name: product, dtype: int64

In [6]:
df_full = df[df["product"] == "Full Season"]

In [8]:
df_full["product"].value_counts()

Full Season    319
Name: product, dtype: int64

In [9]:
df_full["product_current"].value_counts()

Full Season    185
None           118
Half Season     10
Individual       6
Name: product_current, dtype: int64

In [26]:
df_half_for_full = df_full[df_full["product_current"] == "Half Season"]
df_half_for_full.sort_values(by="sascore", ascending=False)

,atp_last,attendance_current,attended_last,attended_prior,clientcode,dimcustomermasterid,distance,events_current,events_last,events_prior,lkupclientid,product_current,product_last,product,sascore,seasonyear,scoredate,spend_current,tenure,volume_current,sends,sends_prior,opens,opens_prior,date_last_send,date_last_touch,date_last_save,newflag,priority
44535,0.00,0.777778,0,54,CANUCKS,118432,33.95,18,0,64,7,Half Season,None,Full Season,0.54,2022,2022-10-19 19:53:13,4568.30,-1,40,25,9,39,22,2022-10-13,NaN,2022-10-19,0,0.22
119063,78.39,0.850000,18,123,CANUCKS,260613,11.48,20,18,126,7,Half Season,None,Full Season,0.50,2022,2022-10-19 19:53:13,4089.67,-1,45,43,20,70,31,2022-10-13,NaN,2022-10-19,0,0.19
109893,152.27,0.277778,1,21,CANUCKS,283521,250.55,18,9,61,7,Half Season,Quarter,Full Season,0.45,2022,2022-10-19 19:53:13,5944.30,-1,36,26,9,11,8,2022-10-13,NaN,2022-10-19,0,0.12
41349,163.63,0.944444,5,55,CANUCKS,94355,15.89,18,9,93,7,Half Season,Quarter,Full Season,0.44,2022,2022-10-19 19:53:13,5944.30,-1,36,40,16,54,38,2022-10-13,NaN,2022-10-19,0,0.13
54905,144.25,0.157895,3,23,CANUCKS,97946,192.36,19,10,74,7,Half Season,Quarter,Full Season,0.42,2022,2022-10-19 19:53:14,5219.90,-1,39,40,17,50,33,2022-10-13,NaN,2022-10-19,0,0.10
211562,116.18,0.500000,7,63,CANUCKS,549078,29.88,18,11,85,7,Half Season,Quarter,Full Season,0.39,2022,2022-10-19 19:53:16,5944.30,-1,38,26,11,36,20,2022-10-13,NaN,2022-10-19,0,0.15
95257,127.44,0.263158,3,8,CANUCKS,213083,9.66,19,3,30,7,Half Season,Mini Pack,Full Season,0.33,2022,2022-10-19 19:53:12,2597.44,-1,49,37,19,59,46,2022-10-13,NaN,2022-10-19,0,0.04
139529,112.58,0.111111,1,26,CANUCKS,355787,71.50,18,10,62,7,Half Season,None,Full Season,0.33,2022,2022-10-19 19:53:14,7193.40,-1,51,36,18,92,33,2022-10-13,NaN,2022-10-19,0,0.05
48034,133.36,0.777778,10,24,CANUCKS,123177,4.33,18,10,28,7,Half Season,Quarter,Full Season,0.32,2022,2022-10-19 19:53:12,5944.30,-1,36,27,9,36,18,2022-10-13,NaN,2022-10-19,0,0.00
210015,106.80,0.631579,4,18,CANUCKS,531364,40.53,19,4,25,7,Half Season,Individual,Full Season,0.23,2022,2022-10-19 19:53:16,4568.30,-1,37,9,7,24,43,2022-10-13,NaN,2022-10-19,0,0.00


In [25]:
df_none_for_full = df_full[df_full["product_current"] == "None"]
df_none_for_full.sort_values(by="sascore", ascending=False)

,atp_last,attendance_current,attended_last,attended_prior,clientcode,dimcustomermasterid,distance,events_current,events_last,events_prior,lkupclientid,product_current,product_last,product,sascore,seasonyear,scoredate,spend_current,tenure,volume_current,sends,sends_prior,opens,opens_prior,date_last_send,date_last_touch,date_last_save,newflag,priority
119367,0.00,0.000000,0,10,CANUCKS,261416,19.24,0,0,11,7,None,None,Full Season,0.72,2022,2022-10-19 19:53:13,0.00,-1,0,1,0,1,0,2022-09-23,NaN,2022-09-22,0,0.40
93273,0.00,0.000000,0,10,CANUCKS,241188,19.24,0,0,11,7,None,None,Full Season,0.72,2022,2022-10-19 19:53:14,0.00,-1,0,1,0,1,0,2022-09-14,NaN,2022-10-13,0,0.39
104008,0.00,0.909091,0,10,CANUCKS,224103,18.60,11,0,11,7,None,None,Full Season,0.68,2022,2022-10-19 19:53:14,2144.50,-1,22,32,20,37,29,2022-10-14,NaN,2022-10-14,0,0.37
193476,0.00,1.000000,0,3,CANUCKS,489627,10.49,3,0,3,7,None,None,Full Season,0.64,2022,2022-10-19 19:53:16,0.00,-1,4,12,10,0,0,2022-10-12,NaN,2022-10-19,0,0.34
275853,0.00,1.000000,0,3,CANUCKS,2335140,10.49,3,0,3,7,None,None,Full Season,0.64,2022,2022-10-19 19:53:16,0.00,-1,5,3,2,1,0,2022-10-13,NaN,2022-10-19,0,0.30
136860,0.00,1.000000,0,3,CANUCKS,333027,10.49,1,0,3,7,None,None,Full Season,0.64,2022,2022-10-19 19:53:13,0.00,-1,2,0,1,0,0,2022-04-27,NaN,2022-10-18,0,0.34
245756,0.00,0.000000,0,3,CANUCKS,637592,10.49,0,0,3,7,None,None,Full Season,0.64,2022,2022-10-19 19:53:17,0.00,-1,0,25,15,3,9,2022-10-14,NaN,2022-10-19,0,0.34
240804,0.00,1.000000,0,3,CANUCKS,609196,10.49,1,0,3,7,None,None,Full Season,0.64,2022,2022-10-19 19:53:16,197.00,-1,2,7,3,9,6,2022-10-13,NaN,2022-10-13,0,0.34
214307,0.00,0.000000,0,3,CANUCKS,584994,10.49,0,0,3,7,None,None,Full Season,0.64,2022,2022-10-19 19:53:17,0.00,-1,0,0,0,0,0,None,NaN,2022-08-11,0,0.37
345789,0.00,1.000000,0,3,CANUCKS,296676171,10.49,3,0,3,7,None,None,Full Season,0.64,2022,2022-10-19 19:53:19,0.00,-1,3,3,2,8,5,2022-10-13,NaN,2022-10-19,0,0.22


In [27]:
df_half_pc = df[df["product_current"] == "Half Season"]
df_half_pc["product"].value_counts()

Half Season    150
None           106
Full Season     10
Quarter          2
Individual       1
Name: product, dtype: int64